# Health Analysis
Done by Isabella Yan (Chisa)

## Question
How do traffic crashes, adult physical inactivity, and Divvy station accessibility relate across Chicago’s community areas, and do neighborhoods with higher physical inactivity experience different crash rates or different access to bike-share infrastructure?

## Sources
The first one I'd like to use is [Chicago Health Atlas](https://chicagohealthatlas.org/indicators/HCSPAP?topic=adult-physical-inactivity-rate) to find the traffic crashes, adult physical inactivity counts, and adult physical inactivity rates by neighborhood. The next one I'd like to use is [Divvy Bikes](https://divvybikes.com/system-data) to get the location of Divvy stops using the latitude and longtitude. Finaly, I'd like to use this [City of Chicago](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Community-Areas-Map/cauq-8yn6) dataset to help map latitude-longitude coordinates to a neighborhood and find the overall density of Divvy stations in each neighborhood.

### Step 1: Clean up data

In [10]:
# load in packages
import pandas as pd
import numpy as np
from shapely import wkt
import geopandas as gpd
from shapely.geometry import Point

# load data from healthdatasets
community_boundary_data = pd.read_csv('./datasets/healthdatasets/Boundaries_-_Community_Areas_20251201.csv')
divvy_data = pd.read_csv('./datasets/healthdatasets/202412-divvy-tripdata.csv')
health_data = pd.read_csv('./datasets/healthdatasets/2024chicagohealthatlas_raw.csv')

In [16]:
#
# 1.1. Clean community area boundaries
#

# rename columns
community_boundary_data = community_boundary_data.rename(columns={
    'AREA_NUMBE': 'community_area_number',
    'COMMUNITY': 'community_area_name',
    'AREA_NUM_1': 'community_area_number_dup',
    'the_geom': 'wkt_geometry',
    'SHAPE_AREA': 'shape_area',
    'SHAPE_LEN': 'shape_length'
})

# standardize community name 
community_boundary_data['community_area_name'] = (
    community_boundary_data['community_area_name'].str.title()
)

# use wkt from shapely to convert the wkt to geometry
community_boundary_data['geometry'] = community_boundary_data['wkt_geometry'].apply(wkt.loads)

# convert to geopanda dataframe 
community_boundary_data = gpd.GeoDataFrame(
    community_boundary_data,
    geometry='geometry',
    crs="EPSG:4326"
)

# extract centroids (useful for lat/lon)
# reproject to Illinois StatePlane (good for Chicago)
boundary_projected = community_boundary_data.to_crs(epsg=3435)

# compute centroids in that CRS
centroids = boundary_projected.centroid

# convert centroids back to WGS84 (lat/lon)
centroids_wgs = centroids.to_crs(epsg=4326)

# store results
community_boundary_data['centroid_lat'] = centroids_wgs.y
community_boundary_data['centroid_lon'] = centroids_wgs.x

#
# 1.2. Clean divvy data
#

# convert all timestamps to datetime objects 
divvy_data['started_at'] = pd.to_datetime(divvy_data['started_at'])
divvy_data['ended_at'] = pd.to_datetime(divvy_data['ended_at'])

# remove invalid rides
divvy_data = divvy_data[
    (divvy_data['ended_at'] > divvy_data['started_at']) &
    (divvy_data['start_lat'].notna()) &
    (divvy_data['start_lng'].notna())
]

# create duration + data features
divvy_data['ride_minutes'] = (divvy_data['ended_at'] - divvy_data['started_at']).dt.total_seconds() / 60
divvy_data['date'] = divvy_data['started_at'].dt.date
divvy_data['hour'] = divvy_data['started_at'].dt.hour
divvy_data['day_of_week'] = divvy_data['started_at'].dt.day_name()

# connect divvy start and end points to lat and lon 
divvy_data['start_point'] = divvy_data.apply(lambda r: Point(r['start_lng'], r['start_lat']), axis=1)
divvy_data['end_point'] = divvy_data.apply(lambda r: Point(r['end_lng'], r['end_lat']), axis=1)


#
# 1.3. Clean health data 
#

# drop metadata rows
health_data = health_data.iloc[4:].reset_index(drop=True)

# rename columns to understand them    
health_data = health_data.rename(columns={
    'Name': 'community_area_name',
    'GEOID': 'community_area_number',
    'HCSPAP_2023-2024': 'physical_inactivity_rate',
    'HCSPAP_2023-2024_moe': 'physical_inactivity_rate_moe',
    'HCSPA_2023-2024': 'physical_inactivity_count',
    'HCSPA_2023-2024_moe': 'physical_inactivity_count_moe',
    'TRC_2024': 'traffic_crashes',
    'TRC_2024_moe': 'traffic_crashes_moe'
})

# standardize the names
health_data['community_area_name'] = (
    health_data['community_area_name'].str.title()
)

# convert geoid to int 
health_data['community_area_number'] = (
    health_data['community_area_number'].astype(int)
)

# drop the layer col
health_data = health_data.drop(columns=['Layer'])


### Step 2. Convert Divvy into a GeoDataframe and Assign them to Neighborhoods to Get Counts

In [ ]:
#
# 2.1. Convert Divvy start points into the geoframe 
#
divvy_gdf = gpd.GeoDataFrame(
    divvy_data,
    geometry=divvy_data['start_point'],
    crs="EPSG:4326"
)

# reproject to match your boundaries
divvy_gdf = divvy_gdf.to_crs(epsg=3435)

# 
# 2.2. Assign each ride's starting neighborhood  
#
divvy_with_area = gpd.sjoin(
    divvy_gdf,
    community_boundary_data[['community_area_number','community_area_name','geometry']],
    how='left',
    predicate='within'
)

### Step 3. Get the Divvy station counts to measure the accessibility in each neighborhood. 

In [ ]:
#
# 3.1. Identify only unique stations
#
unique_stations = divvy_with_area[['start_station_name', 'geometry']].drop_duplicates()

#
# 3.2. Count stations per neighborhood. 
#
station_density = unique_stations.groupby('community_area_number').size().reset_index(name='divvy_station_count')
